# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [4]:
data.race.unique()

array(['w', 'b'], dtype=object)

In [9]:
print('Total Resumes: '+str(len(data)))
print('Black Resumes: '+str(len(data[data['race']=='b'])))
print("callbacks for black-sounding names: ", sum(data[data.race=='b'].call))
print('White Resumes: '+str(len(data[data['race']=='w'])))
print("callbacks for white-sounding names: ", sum(data[data.race=='w'].call))

Total Resumes: 4870
Black Resumes: 2435
callbacks for black-sounding names:  157.0
White Resumes: 2435
callbacks for white-sounding names:  235.0


### 1. What test is appropriate for this problem? Does CLT apply?

<div class="span5 alert alert-success">
<p> Here, we are dealing with two different samples (race = black and race = white) with a value of 0 or 1 for no call-back vs. call-back. We can use a hypothesis test to find out whether race sounding name affects rate of callback. There will be 2 seperate bernoulli distributions for race black and white.
<br><br>We have a nice large sample size here(>30). The data is evenly splitted between black and white records.Also, the observations are independent.
<br><b>So CLT is applicable in this case.</b>
</p>
</div>

### 2. What are the null and alternate hypotheses?

<div class="span5 alert alert-success">
<p> 
    H0: There is <b>NO</b> racial discrimination in the U.S. work labor market i.e. the probability of callback is <b>same</b> for white and black sounding names p_b = p_w
<br>H1: There is racial discrimination in the U.S. work labor market i.e. the probability of callback is <b>NOT Same</b> for black and white sounding names p_b != p_w
</p>
</div>

### 3. Compute margin of error, confidence interval, and p-value. 
#### Try using both the bootstrapping and the frequentist statistical approaches.

In [59]:
w = data[data.race=='w']
b = data[data.race=='b']
#proportion of white people getting a callback
p_w_c = len(w[w.call==1])/len(w)
print("proportion of white people getting a callback: {:3f}".format(p_w_c))
#proportion of black people getting a callback
p_b_c = len(b[b.call==1])/len(b)
print("proportion of black people getting a callback: {:3f}".format(p_b_c))

proportion of white people getting a callback: 0.096509
proportion of black people getting a callback: 0.064476


In [61]:
#bernoulli distribution:
#In a Bernoulli dist., the standard deviation of each sample is equal to the proportion getting a callback 
#times one minus the proportion getting a callback
sigma_w = p_w_c * (1-p_w_c)
sigma_b = p_b_c * (1-p_b_c)
 
#Find the difference between the proportions of black and white people getting a callback
diff = np.round(np.abs(p_w_c - p_b_c),3)
print("The difference in callback rates: ", diff)

# Find the standard error
SE = np.sqrt((sigma_b)/len(b)  + (sigma_w)/len(w))

The difference in callback rates:  0.032


For a 95% confidence interval, use a z* = 1.96. The margin of error (ME) and confidence interval (CI) are:

In [62]:
ME = 1.96 * SE
ME  = np.round(ME, 3)
print ('Margin of Error:', ME, "%")

CI = (diff - ME, diff + ME)
print ('Confidence Interval:', CI)

Margin of Error: 0.015 %
Confidence Interval: (0.017, 0.047)


In [63]:
print ("P-Value = ", 2 * stats.norm.cdf(0, diff, SE))

P-Value =  3.9338756257269395e-05


### 4. Write a story describing the statistical significance in the context or the original problem.

<div class="span5 alert alert-success">
<p> Since the p-value is much smaller than the significance level (0.05), there is convincing evidence to reject the null hypothesis. Thus we have to go with our alternate hypothesis and conclude there was indeed a racial discrimination in the labor market as per the data.
</p>
</div>

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

As per our analysis, we can say with 95% confidence that the number of call backs to white versus black sounding names is statistically significant.
<br><br>But this data set includes a number of other important features like education, years of experience etc. which may paly a very important role in our analysis. Thus we cannot conclude that race/name is the most important factor, unless we analyze the other factors.